In [ ]:
STATE_LIST = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
              'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
              'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
              'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
              'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR']

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

from folktables import ACSDataSource, ACSIncome
import numpy as np
import pandas as pd
from scipy.stats import beta
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import sklearn.preprocessing as preprocessing
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from responsibly.fairness.interventions.threshold import find_thresholds





data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = pd.DataFrame()

for state in STATE_LIST:
    acs_data = pd.concat([acs_data, data_source.get_data(states=[state], download=False)])

features, label, group = ACSIncome.df_to_numpy(acs_data)

In [ ]:
fair_constarint = ['single', 'min_cost', 'independence', 'fnr']
fair = 3

# Define true parameter settings
min_1_group1 = 0
max_1_group1 = 1
min_0_group1 = 0
max_0_group1 = 1
min_1_group0 = 0
max_1_group0 = 1
min_0_group0 = 0
max_0_group0 = 1

reference_quantile_0 = 0.6
reference_quantile_1 = 0.5
exploration_porb_group1 = 1
exploration_porb_group0 = 1
batchsize_init = 6000
batchsize_additional = 2000

# Defining other parameters
TP_group1 = 0
FP_group1 = 0
FN_group1 = 0
TN_group1 = 0
TP_group0 = 0
FP_group0 = 0
FN_group0 = 0
TN_group0 = 0

TP_oracle_group1 = 0 
FP_oracle_group1 = 0
FN_oracle_group1 = 0
TN_oracle_group1 = 0
TP_oracle_group0 = 0 
FP_oracle_group0 = 0
FN_oracle_group0 = 0
TN_oracle_group0 = 0

In [ ]:
features.shape, label.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, label, train_size = 0.99999, shuffle=True, random_state = 0)

classifier = LogisticRegression().fit(X_train, y_train)
prob = classifier.predict_proba(features)[:,1]

original = pd.DataFrame(features)
original["Prob"] = prob
original['Target'] = label
original['Race'] = group
original

# Split four groups data for true parameters mapping
grouphigh_label1 = original[(original["Target"]==True) & (original["Race"]==1)]["Prob"]
grouphigh_label0 = original[(original["Target"]==False) & (original["Race"]==1)]["Prob"]
grouplow_label1 = original[(original["Target"]==True) & (original["Race"]!=1)]["Prob"]
grouplow_label0 = original[(original["Target"]==False) & (original["Race"]!=1)]["Prob"]

# Reindex four groups data
grouphigh_label1.reset_index(inplace=True, drop=True)
grouphigh_label0.reset_index(inplace=True, drop=True)
grouplow_label1.reset_index(inplace=True, drop=True)
grouplow_label0.reset_index(inplace=True, drop=True)

a,b1_true_group1,c,d = beta.fit(grouphigh_label1,floc=min_1_group1,fscale=max_1_group1-min_1_group1)
a,b0_true_group1,c,d = beta.fit(grouphigh_label0,floc=min_0_group1,fscale=max_0_group1-min_0_group1)
a,b1_true_group0,c,d = beta.fit(grouplow_label1,floc=min_1_group0,fscale=max_1_group0-min_1_group0)
a,b0_true_group0,c,d = beta.fit(grouplow_label0,floc=min_0_group0,fscale=max_0_group0-min_0_group0)

x = np.arange(0.01,10,0.01)
def f(alpha):
    return abs(beta.cdf(np.quantile(grouphigh_label1,reference_quantile_1), alpha, b1_true_group1, loc=min_1_group1, scale=max_1_group1-min_1_group1) - reference_quantile_1)
a1_true_group1 = np.argmin(f(x))*0.01

def f(alpha):
    return abs(beta.cdf(np.quantile(grouphigh_label0,reference_quantile_0), alpha, b0_true_group1, loc=min_0_group1, scale=max_0_group1-min_0_group1) - reference_quantile_0)
a0_true_group1 = np.argmin(f(x))*0.01

def f(alpha):
    return abs(beta.cdf(np.quantile(grouplow_label1,reference_quantile_1), alpha, b1_true_group0, loc=min_1_group0, scale=max_1_group0-min_1_group0) - reference_quantile_1)
a1_true_group0 = np.argmin(f(x))*0.01

def f(alpha):
    return abs(beta.cdf(np.quantile(grouplow_label0,reference_quantile_0), alpha, b0_true_group0, loc=min_0_group0, scale=max_0_group0-min_0_group0) - reference_quantile_0)
a0_true_group0 = np.argmin(f(x))*0.01

print (a1_true_group1,b1_true_group1,a0_true_group1,b0_true_group1,
       a1_true_group0,b1_true_group0,a0_true_group0,b0_true_group0)

In [ ]:
# Select first (Initial_fit_portion) rows to find initial assumed distribution
Initial_fit_portion = 0.03
X_train, X_test, y_train, y_test = train_test_split(features, label, train_size = Initial_fit_portion, shuffle=True, random_state = 0)

classifier = LogisticRegression().fit(X_train, y_train)
prob = classifier.predict_proba(features)[:,1]
original["Prob"] = prob
First_n_data = original.head(round(Initial_fit_portion*len(features)))
Initialhigh_label1 = First_n_data[(First_n_data["Target"]==True) & (First_n_data["Race"]==1)]["Prob"]
Initialhigh_label0 = First_n_data[(First_n_data["Target"]==False) & (First_n_data["Race"]==1)]["Prob"]
Initiallow_label1 = First_n_data[(First_n_data["Target"]==True) & (First_n_data["Race"]!=1)]["Prob"]
Initiallow_label0 = First_n_data[(First_n_data["Target"]==False) & (First_n_data["Race"]!=1)]["Prob"]

# Reindex
Initialhigh_label1.reset_index(inplace=True, drop=True)
Initialhigh_label0.reset_index(inplace=True, drop=True)
Initiallow_label1.reset_index(inplace=True, drop=True)
Initiallow_label0.reset_index(inplace=True, drop=True)

# Fix beta parameter
b1_init_group1 = b1_true_group1
b0_init_group1 = b0_true_group1
b1_init_group0 = b1_true_group0
b0_init_group0 = b0_true_group0

b1_group1 = b1_init_group1
b0_group1 = b0_init_group1
b1_group0 = b1_init_group0
b0_group0 = b0_init_group0

# Find initial alpha parameter by fixing beta value
def f(alpha):
    return abs(beta.cdf(np.quantile(Initialhigh_label1,reference_quantile_1), alpha, b1_true_group1, loc=min_1_group1, scale=max_1_group1-min_1_group1) - reference_quantile_1)
a1_init_group1 = np.argmin(f(x))*0.01

def f(alpha):
    return abs(beta.cdf(np.quantile(Initialhigh_label0,reference_quantile_0), alpha, b0_true_group1, loc=min_0_group1, scale=max_0_group1-min_0_group1) - reference_quantile_0)
a0_init_group1 = np.argmin(f(x))*0.01

def f(alpha):
    return abs(beta.cdf(np.quantile(Initiallow_label1,reference_quantile_1), alpha, b1_true_group0, loc=min_1_group0, scale=max_1_group0-min_1_group0) - reference_quantile_1)
a1_init_group0 = np.argmin(f(x))*0.01

def f(alpha):
    return abs(beta.cdf(np.quantile(Initiallow_label0,reference_quantile_0), alpha, b0_true_group0, loc=min_0_group0, scale=max_0_group0-min_0_group0) - reference_quantile_0)
a0_init_group0 = np.argmin(f(x))*0.01

a1_group1 = a1_init_group1
a0_group1 = a0_init_group1
a1_group0 = a1_init_group0
a0_group0 = a0_init_group0
print (a1_group1,b1_group1,a0_group1,b0_group1,
       a1_group0,b1_group0,a0_group0,b0_group0)

In [ ]:
# Find the confusion matrix given threshold
def CM(Y_test,y_pred,threshold):
    # The Confusion Matrix given a threshold
    y_pred = np.where(y_pred>threshold,1,0)
    cm = pd.DataFrame(confusion_matrix(Y_test,y_pred))
    cm.rename(columns={0:'Pred_neg', 1:'Pred_pos'},
         index = {0:'Actual_neg',1:'Actual_pos'},inplace=True)
    cm['Total'] = cm['Pred_neg'] + cm['Pred_pos'] 
    rowsum = cm.sum()
    rowsum.name = 'Total'
    cm = cm.append(rowsum.transpose())
    
    # TP/TN/FP/FN/TPR/FPR
    P = cm['Total']['Actual_pos']
    N = cm['Total']['Actual_neg']
    TP = cm['Pred_pos']['Actual_pos']
    TN = cm['Pred_neg']['Actual_neg']
    FP = cm['Pred_pos']['Actual_neg']
    FN = cm['Pred_neg']['Actual_pos']
    TPR = np.round(TP/P,2)
    FPR = np.round(FP/N,2)
    return(cm, TPR, FPR)

# Define the cost matrix
COST_MATRIX = [[0, -3/6],
               [0,  3/6]]

In [ ]:
# Find fair classifier for the inital training set

# Find the parameters proportions, base_rate and base_rates 
proportions = {'White': (len(Initialhigh_label1) + len(Initialhigh_label0))/(len(Initialhigh_label1) + len(Initialhigh_label0) + len(Initiallow_label1) + len(Initiallow_label0)), 
               'Non-White': 1 - (len(Initialhigh_label1) + len(Initialhigh_label0))/(len(Initialhigh_label1) + len(Initialhigh_label0) + len(Initiallow_label1) + len(Initiallow_label0))}
base_rate = (len(Initialhigh_label1) + len(Initiallow_label1))/(len(Initialhigh_label1) + len(Initialhigh_label0) + len(Initiallow_label1) + len(Initiallow_label0))
d = {'White': len(Initialhigh_label1)/(len(Initialhigh_label1) + len(Initialhigh_label0)), 'Non-White': len(Initiallow_label1)/(len(Initiallow_label1) + len(Initiallow_label0))}
base_rates = pd.Series(data = d, index = ['White','Non-White'])

# Find the ROC curve by construting confusion matrix 
yprob1 = First_n_data[First_n_data["Race"]==1]["Prob"]
yprob0 = First_n_data[First_n_data["Race"]!=1]["Prob"]
  
TPR1 = []
FPR1 = []
threshold1 = []
for threshold in range(201,0,-1):
    cm, TPR, FPR = CM(First_n_data[First_n_data["Race"]==1]["Target"],yprob1,threshold/200)
    TPR1 = TPR1 + [TPR]
    FPR1 = FPR1 + [FPR]
    threshold1 = threshold1 + [threshold/200]
TPR1 = np.array(TPR1)
FPR1 = np.array(FPR1)
threshold1 = np.array(threshold1)

TPR0 = []
FPR0 = []
threshold0 = []
for threshold in range(201,0,-1):
    cm, TPR, FPR = CM(First_n_data[First_n_data["Race"]!=1]["Target"],yprob0,threshold/200)
    TPR0 = TPR0 + [TPR]
    FPR0 = FPR0 + [FPR]
    threshold0 = threshold0 + [threshold/200]
TPR0 = np.array(TPR0)
FPR0 = np.array(FPR0)
threshold0 = np.array(threshold0)

# Find the parameter rocs 
rocs = {'White': (FPR1, TPR1, threshold1), 'Non-White': (FPR0, TPR0,threshold0)}

# Find the initial fairness classifier_1 and classifier_2 
thresholds_data = find_thresholds(rocs, proportions, base_rate, base_rates, COST_MATRIX)
if fair != 0:
    classifier_group1 = thresholds_data[fair_constarint[fair]][0]['White']
    classifier_group0 = thresholds_data[fair_constarint[fair]][0]['Non-White']
else: 
    classifier_group1 = thresholds_data[fair_constarint[fair]][0]
    classifier_group0 = thresholds_data[fair_constarint[fair]][0]
#print (classifier_group1, classifier_group0)

In [ ]:
# Find oracle fair classifier for the entire set

# Find the parameters proportions, base_rate and base_rates 
proportions = {'White': (len(grouphigh_label1) + len(grouphigh_label0))/(len(grouphigh_label1) + len(grouphigh_label0) + len(grouplow_label1) + len(grouplow_label1)), 
               'Non-White': 1 - (len(grouphigh_label1) + len(grouphigh_label0))/(len(grouphigh_label1) + len(grouphigh_label0) + len(grouplow_label1) + len(grouplow_label1))}
base_rate = (len(grouphigh_label1) + len(grouplow_label1))/(len(grouphigh_label1) + len(grouphigh_label0) + len(grouplow_label1) + len(grouplow_label1))
d = {'White': len(grouphigh_label1)/(len(grouphigh_label1) + len(grouphigh_label0)), 'Non-White': len(grouplow_label1)/(len(grouplow_label1) + len(grouplow_label0))}
base_rates = pd.Series(data = d, index = ['White','Non-White'])

# Find the ROC curve by construting confusion matrix 
yprob1 = original[original["Race"]==1]["Prob"]
yprob0 = original[original["Race"]!=1]["Prob"]

TPR1 = []
FPR1 = []
threshold1 = []
for threshold in range(201,0,-1):
    cm, TPR, FPR = CM(original[original["Race"]==1]["Target"],yprob1,threshold/200)
    TPR1 = TPR1 + [TPR]
    FPR1 = FPR1 + [FPR]
    threshold1 = threshold1 + [threshold/200]
TPR1 = np.array(TPR1)
FPR1 = np.array(FPR1)
threshold1 = np.array(threshold1)

TPR0 = []
FPR0 = []
threshold0 = []
for threshold in range(201,0,-1):
    cm, TPR, FPR = CM(original[original["Race"]!=1]["Target"],yprob0,threshold/200)
    TPR0 = TPR0 + [TPR]
    FPR0 = FPR0 + [FPR]
    threshold0 = threshold0 + [threshold/200]
TPR0 = np.array(TPR0)
FPR0 = np.array(FPR0)
threshold0 = np.array(threshold0)

# Find the parameter rocs 
rocs = {'White': (FPR1, TPR1, threshold1), 'Non-White': (FPR0, TPR0,threshold0)}

# Find the initial fairness classifier_1 and classifier_2 
thresholds_data = find_thresholds(rocs, proportions, base_rate, base_rates, COST_MATRIX)
if fair != 0:
    classifier_oracle_value_1 = thresholds_data[fair_constarint[fair]][0]['White']
    classifier_oracle_value_0 = thresholds_data[fair_constarint[fair]][0]['Non-White']
else: 
    classifier_oracle_value_1 = thresholds_data[fair_constarint[fair]][0]
    classifier_oracle_value_0 = thresholds_data[fair_constarint[fair]][0]
#print (classifier_oracle_value_1, classifier_oracle_value_0)

In [ ]:
# Find the LB_1 and UB_1 with fair classifier(Using alpha = 60 for label 0 and median for label 1)
temp = 2*beta.cdf(beta.ppf(reference_quantile_0,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1),a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1) -\
            beta.cdf(classifier_group1,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1)
LB_group1 = max(min_0_group1, float(beta.ppf(temp,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1)))

temp = 2*beta.cdf(beta.ppf(reference_quantile_1,a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1),a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1) -\
            beta.cdf(LB_group1,a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1)
UB_group1 = min(max_1_group1, float(beta.ppf(temp,a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1)))

# Find the LB_0 and UB_0 with fair classifier(Using alpha = 60 for label 0 and median for label 1)
temp = 2*beta.cdf(beta.ppf(reference_quantile_0,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0),a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0) -\
            beta.cdf(classifier_group0,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0)
LB_group0 = max(min_0_group0, float(beta.ppf(temp,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0)))

temp = 2*beta.cdf(beta.ppf(reference_quantile_1,a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0),a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0) -\
            beta.cdf(LB_group0,a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0)
UB_group0 = min(max_1_group0, float(beta.ppf(temp,a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0)))
print (LB_group1, UB_group1, LB_group0, UB_group0)

In [ ]:
from sklearn.utils import shuffle
data_1_group1 = Initialhigh_label1
data_0_group1 = Initialhigh_label0
data_1_group0 = Initiallow_label1
data_0_group0 = Initiallow_label0

Rest_data = original.tail(round((1-Initial_fit_portion)*len(features)))[["Prob","Target","Race"]]
Rest_data.reset_index(inplace=True, drop=True)

info = pd.DataFrame(columns = ['True Label','Values','Decision','RP_1_group1','RP_0_group1',
                                      'Classifier_group1','Explore_Prob_group1','RP_1_group0','RP_0_group0',
                                      'Classifier_group0','Explore_Prob_group0','Alpha_1_group1','Alpha_0_group1',
                                       'Alpha_1_group0','Alpha_0_group0','Regret','Race'])
info[["Values","True Label","Race"]] = Rest_data
info = shuffle(info)
info.reset_index(inplace=True, drop=True)

In [ ]:
info

In [ ]:
i = 0
# Create loop for updating
while (i <= len(info)-1):
    
    if i<= (len(grouphigh_label1) + len(grouphigh_label0) + len(grouplow_label1) + len(grouplow_label1)):
        batchsize = batchsize_init
    else: 
        batchsize = batchsize_additional
        
    data_1_trun_group1 = []
    data_0_trun_group1 = []
    data_1_trun_group0 = []
    data_0_trun_group0 = []
    
    # find quantile of reference point including label 0 tail for advantage group
    portion_right0_group1 = (beta.sf(beta.ppf(reference_quantile_0,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1),
                                    a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1)
                     /beta.sf(LB_group1,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1))
    portion_left0_group1 = 1 - portion_right0_group1
    
    portion_right1_group1 = (beta.sf(beta.ppf(reference_quantile_1,a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1),
                                    a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1)
                     /beta.sf(LB_group1,a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1))
    portion_left1_group1 = 1 - portion_right1_group1
    
    # find quantile of reference point including label 0 tail for disadvantage group
    portion_right0_group0 = (beta.sf(beta.ppf(reference_quantile_0,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0),
                                    a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0)
                     /beta.sf(LB_group0,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0))
    portion_left0_group0 = 1 - portion_right0_group0
    
    portion_right1_group0 = (beta.sf(beta.ppf(reference_quantile_1,a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0),
                                    a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0)
                     /beta.sf(LB_group0,a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0))
    portion_left1_group0 = 1 - portion_right1_group0
    
    k1_group1 = len(data_1_trun_group1)
    k0_group1 = len(data_0_trun_group1)
    k1_group0 = len(data_1_trun_group0)
    k0_group0 = len(data_0_trun_group0)
    #loop into each batch
    while ((min(k1_group1, k0_group1, k1_group0, k0_group0) <= batchsize) & (i <= len(info)-1)):
        
        if info.loc[i,'Race'] == 1:
        
            # Make decisions
            if (info.loc[i,'Values'] >= classifier_group1) & (info.loc[i,'Values']<=UB_group1):
                info.loc[i,'Decision'] = 1
            elif (info.loc[i,'Values'] < classifier_group1) & (info.loc[i,'Values']>=LB_group1) & (np.random.uniform(size=1)<=exploration_porb_group1):
                info.loc[i,'Decision'] = 1
            elif (info.loc[i,'Values'] >= classifier_group1) & (info.loc[i,'Values']>UB_group1):
                info.loc[i,'Decision'] = 2
            elif (info.loc[i,'Values'] < classifier_group1) & (info.loc[i,'Values']<LB_group1):
                info.loc[i,'Decision'] = 3
            else:
                info.loc[i,'Decision'] = 0  

            # Record confusion matrix
            if (info.loc[i,'True Label']==1) & ((info.loc[i,'Decision']==1) or (info.loc[i,'Decision']==2)):
                TP_group1 += 1
            elif (info.loc[i,'True Label']==1) & ((info.loc[i,'Decision']==0) or (info.loc[i,'Decision']==3)):
                FN_group1 += 1
            elif (info.loc[i,'True Label']==0) & ((info.loc[i,'Decision']==0) or (info.loc[i,'Decision']==3)):
                TN_group1 += 1
            else:
                FP_group1 += 1

            # Record oracle matrix
            if (info.loc[i,'Values'] >= classifier_oracle_value_1) & (info.loc[i,'True Label']==1):
                TP_oracle_group1 += 1
            elif (info.loc[i,'Values'] >= classifier_oracle_value_1) & (info.loc[i,'True Label']==0):
                FP_oracle_group1 += 1
            elif (info.loc[i,'Values'] < classifier_oracle_value_1) & (info.loc[i,'True Label']==1):
                FN_oracle_group1 += 1
            else:
                TN_oracle_group1 += 1


            # Record label 1 information for updating
            if (info.loc[i,'True Label']==1) & ((info.loc[i,'Decision']==1) or (info.loc[i,'Decision']==2)):
                data_1_group1 = np.append(data_1_group1, info.loc[i,'Values'])
                if (((info.loc[i,'Values'] < classifier_group1) & (info.loc[i,'Decision']==1)) 
                   or ((info.loc[i,'Values'] >= classifier_group1) & (info.loc[i,'Decision']==1) & (np.random.uniform(size=1)<=exploration_porb_group1))):
                    data_1_trun_group1 = np.append(data_1_trun_group1, info.loc[i,'Values'])
            else:
                data_1_group1 = data_1_group1 
                data_1_trun_group1 = data_1_trun_group1


            # Record label 0 information for updating
            if (info.loc[i,'True Label']==0) & ((info.loc[i,'Decision']==1) or (info.loc[i,'Decision']==2)):
                data_0_group1 = np.append(data_0_group1, info.loc[i,'Values'])
                if (((info.loc[i,'Values'] < classifier_group1) & (info.loc[i,'Decision']==1)) 
                    or ((info.loc[i,'Values'] >= classifier_group1) & (np.random.uniform(size=1)<=exploration_porb_group1))):
                    data_0_trun_group1 = np.append(data_0_trun_group1, info.loc[i,'Values'])
            else:
                data_0_group1 = data_0_group1 
                data_0_trun_group1 = data_0_trun_group1
       
        else: # for non-white group
            
            # Make decisions
            if (info.loc[i,'Values'] >= classifier_group0) & (info.loc[i,'Values']<=UB_group0):
                info.loc[i,'Decision'] = 1
            elif (info.loc[i,'Values'] < classifier_group0) & (info.loc[i,'Values']>=LB_group0) & (np.random.uniform(size=1)<=exploration_porb_group0):
                info.loc[i,'Decision'] = 1
            elif (info.loc[i,'Values'] >= classifier_group0) & (info.loc[i,'Values']>UB_group0):
                info.loc[i,'Decision'] = 2
            elif (info.loc[i,'Values'] < classifier_group0) & (info.loc[i,'Values']<LB_group0):
                info.loc[i,'Decision'] = 3
            else:
                info.loc[i,'Decision'] = 0  

            # Record confusion matrix
            if (info.loc[i,'True Label']==1) & ((info.loc[i,'Decision']==1) or (info.loc[i,'Decision']==2)):
                TP_group0 += 1
            elif (info.loc[i,'True Label']==1) & ((info.loc[i,'Decision']==0) or (info.loc[i,'Decision']==3)):
                FN_group0 += 1
            elif (info.loc[i,'True Label']==0) & ((info.loc[i,'Decision']==0) or (info.loc[i,'Decision']==3)):
                TN_group0 += 1
            else:
                FP_group0 += 1

            # Record oracle matrix
            if (info.loc[i,'Values'] >= classifier_oracle_value_0) & (info.loc[i,'True Label']==1):
                TP_oracle_group0 += 1
            elif (info.loc[i,'Values'] >= classifier_oracle_value_0) & (info.loc[i,'True Label']==0):
                FP_oracle_group0 += 1
            elif (info.loc[i,'Values'] < classifier_oracle_value_0) & (info.loc[i,'True Label']==1):
                FN_oracle_group0 += 1
            else:
                TN_oracle_group0 += 1


            # Record label 1 information for updating
            if (info.loc[i,'True Label']==1) & ((info.loc[i,'Decision']==1) or (info.loc[i,'Decision']==2)):
                data_1_group0 = np.append(data_1_group0, info.loc[i,'Values'])
                if (((info.loc[i,'Values'] < classifier_group0) & (info.loc[i,'Decision']==1)) 
                   or ((info.loc[i,'Values'] >= classifier_group0) & (info.loc[i,'Decision']==1) & (np.random.uniform(size=1)<=exploration_porb_group0))):
                    data_1_trun_group0 = np.append(data_1_trun_group0, info.loc[i,'Values'])
            else:
                data_1_group0 = data_1_group0 
                data_1_trun_group0 = data_1_trun_group0


            # Record label 0 information for updating
            if (info.loc[i,'True Label']==0) & ((info.loc[i,'Decision']==1) or (info.loc[i,'Decision']==2)):
                data_0_group0 = np.append(data_0_group0, info.loc[i,'Values'])
                if (((info.loc[i,'Values'] < classifier_group0) & (info.loc[i,'Decision']==1)) 
                    or ((info.loc[i,'Values'] >= classifier_group0) & (np.random.uniform(size=1)<=exploration_porb_group0))):
                    data_0_trun_group0 = np.append(data_0_trun_group0, info.loc[i,'Values'])
            else:
                data_0_group0 = data_0_group0 
                data_0_trun_group0 = data_0_trun_group0
            
        k1_group1 = len(data_1_trun_group1)
        k0_group1 = len(data_0_trun_group1)
        k1_group0 = len(data_1_trun_group0)
        k0_group0 = len(data_0_trun_group0)
        i = i + 1
    
    print (k1_group1, k0_group1, k1_group0, k0_group0)
    # Record new mean value/RP and classifier
    info.loc[i-1,'RP_1_group1'] = np.quantile(data_1_trun_group1, portion_left1_group1)  #np.median(data_1_trun_group1) 
    info.loc[i-1,'RP_0_group1'] =  np.quantile(data_0_trun_group1, portion_left0_group1) 
    info.loc[i-1,'Classifier_group1'] = classifier_group1
    info.loc[i-1,'Regret'] = (FP_group1 + FN_group1)-(FP_oracle_group1 + FN_oracle_group1)
    info.loc[i-1,'Explore_Prob_group1'] = exploration_porb_group1
    
    info.loc[i-1,'RP_1_group0'] = np.quantile(data_1_trun_group0, portion_left1_group0)   #np.median(data_1_trun_group0)  
    info.loc[i-1,'RP_0_group0'] =  np.quantile(data_0_trun_group0, portion_left0_group0) 
    info.loc[i-1,'Classifier_group0'] = classifier_group0
    info.loc[i-1,'Explore_Prob_group0'] = exploration_porb_group0
    
    x = np.arange(0.01,10,0.01)
    # Reassign new values to parameters
    def f(alpha):
        return abs(beta.cdf(info.loc[i-1,'RP_1_group1'], alpha, b1_group1, loc=min_1_group1, scale=max_1_group1-min_1_group1) - reference_quantile_1)
    info.loc[i-1,'Alpha_1_group1'] = np.argmin(f(x))*0.01
    
    def f(alpha):
        return abs(beta.cdf(info.loc[i-1,'RP_0_group1'], alpha, b0_group1, loc=min_0_group1, scale=max_0_group1-min_0_group1) - reference_quantile_0)
    info.loc[i-1,'Alpha_0_group1'] = np.argmin(f(x))*0.01
    
    def f(alpha):
        return abs(beta.cdf(info.loc[i-1,'RP_1_group0'], alpha, b1_group0, loc=min_1_group0, scale=max_1_group0-min_1_group0) - reference_quantile_1)
    info.loc[i-1,'Alpha_1_group0'] = np.argmin(f(x))*0.01
    
    def f(alpha):
        return abs(beta.cdf(info.loc[i-1,'RP_0_group0'], alpha, b0_group0, loc=min_0_group0, scale=max_0_group0-min_0_group0) - reference_quantile_0)
    info.loc[i-1,'Alpha_0_group0'] = np.argmin(f(x))*0.01
    
    a1_group1 = info.loc[i-1,'Alpha_1_group1']
    a0_group1 = info.loc[i-1,'Alpha_0_group1']
    a1_group0 = info.loc[i-1,'Alpha_1_group0']
    a0_group0 = info.loc[i-1,'Alpha_0_group0']
    
    
    
    # Update fair classifiers, UB and LB
    # Find the parameters proportions, base_rate and base_rates 
    proportions = {'White': (len(data_1_group1) + len(data_0_group1))/(len(data_1_group1) + len(data_0_group1) + len(data_1_group0) + len(data_0_group0)), 
                   'Non-White': 1 - (len(data_1_group1) + len(data_0_group1))/(len(data_1_group1) + len(data_0_group1) + len(data_1_group0) + len(data_0_group0))}
    base_rate = (len(data_1_group1) + len(data_1_group0))/(len(data_1_group1) + len(data_0_group1) + len(data_1_group0) + len(data_0_group0))
    d = {'White': len(data_1_group1)/(len(data_1_group1) + len(data_0_group1)), 'Non-White': len(data_1_group0)/(len(data_1_group0) + len(data_0_group0))}
    base_rates = pd.Series(data = d, index = ['White','Non-White'])

    # Find the ROC curve by construting confusion matrix 
    yprob1 = np.append(data_1_group1, data_0_group1)
    yprob0 = np.append(data_1_group0, data_0_group0)

    TPR1 = []
    FPR1 = []
    threshold1 = []
    for threshold in range(201,0,-1):
        cm, TPR, FPR = CM(np.append(np.ones(len(data_1_group1)),np.zeros(len(data_0_group1))),yprob1,threshold/200)
        TPR1 = TPR1 + [TPR]
        FPR1 = FPR1 + [FPR]
        threshold1 = threshold1 + [threshold/200]
    TPR1 = np.array(TPR1)
    FPR1 = np.array(FPR1)
    threshold1 = np.array(threshold1)

    TPR0 = []
    FPR0 = []
    threshold0 = []
    for threshold in range(201,0,-1):
        cm, TPR, FPR = CM(np.append(np.ones(len(data_1_group0)),np.zeros(len(data_0_group0))),yprob0,threshold/200)
        TPR0 = TPR0 + [TPR]
        FPR0 = FPR0 + [FPR]
        threshold0 = threshold0 + [threshold/200]
    TPR0 = np.array(TPR0)
    FPR0 = np.array(FPR0)
    threshold0 = np.array(threshold0)

    # Find the parameter rocs 
    rocs = {'White': (FPR1, TPR1, threshold1), 'Non-White': (FPR0, TPR0,threshold0)}

    # Find the initial fairness classifier_1 and classifier_0 
    thresholds_data = find_thresholds(rocs, proportions, base_rate, base_rates, COST_MATRIX)
    if fair != 0:
        classifier_group1 = thresholds_data[fair_constarint[fair]][0]['White']
        classifier_group0 = thresholds_data[fair_constarint[fair]][0]['Non-White']
    else: 
        classifier_group1 = thresholds_data[fair_constarint[fair]][0]
        classifier_group0 = thresholds_data[fair_constarint[fair]][0]

    temp = 2*beta.cdf(beta.ppf(reference_quantile_0,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1),a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1) -\
            beta.cdf(classifier_group1,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1)
    LB_group1 = max(min_0_group1, float(beta.ppf(temp,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1)))

    temp = 2*beta.cdf(beta.ppf(reference_quantile_1,a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1),a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1) -\
            beta.cdf(LB_group1,a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1)
    UB_group1 = min(max_1_group1, float(beta.ppf(temp,a1_group1,b1_group1,loc=min_1_group1, scale=max_1_group1-min_1_group1)))
    

    temp = 2*beta.cdf(beta.ppf(reference_quantile_0,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0),a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0) -\
            beta.cdf(classifier_group0,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0)
    LB_group0 = max(min_0_group0, float(beta.ppf(temp,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0)))

    temp = 2*beta.cdf(beta.ppf(reference_quantile_1,a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0),a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0) -\
            beta.cdf(LB_group0,a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0)
    UB_group0 = min(max_1_group0, float(beta.ppf(temp,a1_group0,b1_group0,loc=min_1_group0, scale=max_1_group0-min_1_group0)))
    
    # Update exploration probability for advantage group
    theoretical_value_group1 = beta.sf(classifier_group1,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1) \
                                /beta.sf(LB_group1,a0_group1,b0_group1,loc=min_0_group1, scale=max_0_group1-min_0_group1)
    experiment_value_group1 = len(data_0_trun_group1[data_0_trun_group1>=classifier_group1])/len(data_0_trun_group1[data_0_trun_group1>=LB_group1])
    diff_group1 = abs(theoretical_value_group1 - experiment_value_group1)
     
    exploration_porb_group1 = 1 - 0.1*(i//(len(info)/10))
    # Update exploration probability for disadvantage group
    theoretical_value_group0 = beta.sf(classifier_group0,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0) \
                                /beta.sf(LB_group0,a0_group0,b0_group0,loc=min_0_group0, scale=max_0_group0-min_0_group0)
    experiment_value_group0 = len(data_0_trun_group0[data_0_trun_group0>=classifier_group0])/len(data_0_trun_group0[data_0_trun_group0>=LB_group0])
    diff_group0 = abs(theoretical_value_group0 - experiment_value_group0)
    exploration_porb_group0 = 1 - 0.1*(i//(len(info)/10))